In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# Define the new file path
original_stopid_lookup_file_directory = r'_Original_StopId_Lookup\MBTA_Stopid_lookup.csv'
df_original_stopid_lookup = pd.read_csv(original_stopid_lookup_file_directory)

In [3]:
gtfs_base_directory = Path(r'..\1--Original_2018_GTFS_Recap_Data\GTFS_Recap_-_Fall_2018')


In [4]:
# Define the base directory where your GTFS data is located
# Adjust this path based on where your script is relative to the GTFS data
# '..' goes up one directory from the current script's location.

print(f"Attempting to read GTFS files from: {gtfs_base_directory.resolve()}")

# Define the name of the GTFS file containing stop information
stops_file_name = 'stops.txt'

try:
    # Check if the base directory exists
    if not gtfs_base_directory.exists():
        print(f"Error: The directory '{gtfs_base_directory.resolve()}' does not exist.")
        print("Please ensure the path is correct and the directory is accessible.")
    elif not gtfs_base_directory.is_dir():
        print(f"Error: The path '{gtfs_base_directory.resolve()}' is not a directory.")
        print("Please ensure the path points to the GTFS data directory.")
    else:
        # Construct the full path to stops.txt
        stops_file_path = gtfs_base_directory / stops_file_name

        if stops_file_path.is_file():
            print(f"\nFound '{stops_file_name}' at: {stops_file_path}")
            # Read the stops.txt file
            df_stops = pd.read_csv(stops_file_path)

            # Check if 'stop_id' column exists
            if 'stop_id' in df_stops.columns:
                print("\nSuccessfully loaded stops.txt. Here are the first 10 unique stop_ids:")
                # Get unique stop_ids and display the first few
                unique_stop_ids = df_stops['stop_id'].unique()
                for stop_id in unique_stop_ids[:10]:
                    print(stop_id)
                if len(unique_stop_ids) > 10:
                    print(f"... and {len(unique_stop_ids) - 10} more unique stop_ids.")

                print(f"\nTotal number of unique stop_ids found: {len(unique_stop_ids)}")

                # If you want to see the first few rows of the entire stops DataFrame:
                print("\nFirst 5 rows of the stops DataFrame:")
                print(df_stops.head().to_markdown(index=False, numalign="left", stralign="left"))
            else:
                print(f"Error: 'stop_id' column not found in '{stops_file_name}'.")
                print("Available columns are:", df_stops.columns.tolist())
        else:
            print(f"Error: '{stops_file_name}' not found directly in '{gtfs_base_directory}'.")
            print("Please ensure you have unzipped the GTFS file and that 'stops.txt' is directly")
            print(f"under '{gtfs_base_directory.name}' or adjust the 'stops_file_path' if it's in a subfolder.")

            # Optional: You can uncomment the following to list files in the directory
            # print("\nFiles found in the directory:")
            # for item in gtfs_base_directory.iterdir():
            #     print(item.name)


except FileNotFoundError:
    print(f"An unexpected FileNotFoundError occurred. Please double-check the base directory path: {gtfs_base_directory.resolve()}")
except Exception as e:
    print(f"An error occurred: {e}")

Attempting to read GTFS files from: J:\Shared drives\TMD_TSA\Projects\STOPS\Input Data\MBTA_GTFS\MBTA_2018_GTFS\_DataProcessing\1--Original_2018_GTFS_Recap_Data\GTFS_Recap_-_Fall_2018

Found 'stops.txt' at: ..\1--Original_2018_GTFS_Recap_Data\GTFS_Recap_-_Fall_2018\stops.txt

Successfully loaded stops.txt. Here are the first 10 unique stop_ids:
1
10
10000
10003
10005
10006
10007
10008
10009
1001
... and 9227 more unique stop_ids.

Total number of unique stop_ids found: 9237

First 5 rows of the stops DataFrame:
| stop_id   | stop_code   | stop_name                     | stop_desc   | platform_code   | platform_name   | stop_lat   | stop_lon   | zone_id   | stop_address   | stop_url                         | level_id   | location_type   | parent_station   | wheelchair_boarding   |
|:----------|:------------|:------------------------------|:------------|:----------------|:----------------|:-----------|:-----------|:----------|:---------------|:---------------------------------|:---------

In [5]:
# Create a list to hold the match information
matches = []

if not gtfs_base_directory.exists():
    print(f"Directory not found: {gtfs_base_directory.resolve()}")
elif not gtfs_base_directory.is_dir():
    print(f"The path is not a directory: {gtfs_base_directory.resolve()}")
else:
    print("Listing files with a column name containing 'stop*id' and collecting corresponding stop id column(s):")
    for file in gtfs_base_directory.glob("*.txt"):
        try:
            # Read only a few rows to check for the header
            df_temp = pd.read_csv(file, nrows=5)
            # Filter columns using regex that matches any occurrence of 'stop' then any characters then 'id'
            filtered_cols = df_temp.filter(regex=".*stop.*id.*") # (regex=".*stop_id.*")
            if not filtered_cols.empty:
                cols_str = ", ".join(filtered_cols.columns)
                print(f" - {file.name} with stop id column(s): {cols_str}")
                # Append file name and its column(s) to the matches list
                matches.append({"file_name": file.name, "stop_id_columns": cols_str})
        except Exception as e:
            print(f"Skipping {file.name} (error: {e})")

# Create a dataframe from the matching files info
df_matched = pd.DataFrame(matches)
df_matched
df_matched.to_csv("matched_files_with_stop_id_columns.csv", index=False)

Listing files with a column name containing 'stop*id' and collecting corresponding stop id column(s):
 - stops.txt with stop id column(s): stop_id
 - facilities.txt with stop id column(s): stop_id
 - pathways.txt with stop id column(s): from_stop_id, to_stop_id
 - transfers.txt with stop id column(s): from_stop_id, to_stop_id
 - stop_times.txt with stop id column(s): stop_id


In [6]:
# Collect all stop IDs from the files specified in df_matched
all_stop_ids = set()

for _, row in df_matched.iterrows():
    file_name = row["file_name"]
    stop_id_columns_str = row["stop_id_columns"]
    # Split the string by comma and strip whitespace to get individual column names
    stop_columns = [col.strip() for col in stop_id_columns_str.split(",")]
    
    # Construct the file path using gtfs_base_directory
    file_path = gtfs_base_directory / file_name
    if not file_path.is_file():
        print(f"File {file_name} not found at {file_path}.")
        continue

    try:
        # Read the file as string so that we compare the stop ids consistently
        df_file = pd.read_csv(file_path, dtype=str)
    except Exception as e:
        print(f"Error reading {file_name}: {e}")
        continue

    for col in stop_columns:
        if col in df_file.columns:
            all_stop_ids.update(df_file[col].dropna().unique())
        else:
            print(f"Column {col} not found in {file_name}.")

# Convert stop ids in df_original_stopid_lookup to a set (as strings)
original_stop_ids = set(df_original_stopid_lookup["stop_id"].astype(str).dropna().unique())

# Identify stop ids that appear in the files but not in df_original_stopid_lookup
missing_stop_ids = all_stop_ids - original_stop_ids

print("Stop IDs found in the matched files but not in df_original_stopid_lookup:")
sorted_missing_stop_ids = sorted(missing_stop_ids)
print(sorted_missing_stop_ids)

Stop IDs found in the matched files but not in df_original_stopid_lookup:
['1001', '10134', '1028', '1031', '10420', '10426', '1044', '10832', '1087', '1093', '1095', '1097', '1098', '109800', '109803', '109807', '109834', '109837', '109840', '109843', '109847', '109850', '109856', '109859', '109863', '109868', '109895', '109902', '109905', '109906', '109914', '109915', '1107', '1109', '11111', '111219', '111220', '111222', '111223', '111224', '1113', '111667', '111678', '111691', '1117', '111705', '111756', '111781', '1118', '111834', '111842', '111843', '1119', '111997', '1120', '112011', '112019', '112029', '112037', '112046', '112068', '112094', '1121', '112112', '112113', '112116', '112121', '112129', '112131', '11218', '112196', '1122', '11243', '11245', '1125', '1143', '1146', '114886', '114887', '114888', '114893', '114895', '114896', '115', '11532', '11609', '11673', '1197', '119863', '1199', '119913', '1200', '1208', '12081', '1210', '121018', '1212', '12146', '12148', '12149

In [7]:
import numpy as np

# Make a copy of the original lookup dataframe
df_updated_stopid_lookup = df_original_stopid_lookup.copy()

# Create a mapping from stop_id to stop_name using information from stops.txt (df_stops)
stop_name_map = df_stops.set_index("stop_id")["stop_name"].to_dict()

# For each missing stop_id in sorted_missing_stop_ids, get the corresponding stop_name (or NaN if not found)
new_stop_names = [stop_name_map.get(sid, np.nan) for sid in sorted_missing_stop_ids]

# Create new rows with the stop_name populated
new_rows = pd.DataFrame({
    'StopNo.': np.nan,
    'stop_id': sorted_missing_stop_ids,
    'stop_name': new_stop_names,
    'lat/long': np.nan,
    'Unnamed: 4': np.nan,
    'stop_id_update': [f'C{n:04d}' for n in range(1, len(sorted_missing_stop_ids) + 1)],
    'stop_name.1': np.nan
})

# Append the new rows to the existing lookup table copy
df_updated_stopid_lookup = pd.concat([df_updated_stopid_lookup, new_rows], ignore_index=True)

df_updated_stopid_lookup

,StopNo.,stop_id,stop_name,lat/long,Unnamed: 4,stop_id_update,stop_name.1
0,------------,------------,------------,------------,NaN,------------,------------
1,Stp# 1,1,Washington St opp Ruggles St,"42.330957, -71.082754",>>>>,A1,Washington St opp Ruggles St
2,Stp# 2,10,Theo Glynn Way @ Newmarket Sq,"42.330555, -71.068787",>>>>,A9,Theo Glynn Way @ Newmarket Sq
3,Stp# 3,10000,Tremont St opp Temple Pl,"42.355692, -71.062911",>>>>,A5337,Tremont St opp Temple Pl
4,Stp# 4,10003,Albany St opp Randall St,"42.331591, -71.076237",>>>>,A5338,Albany St opp Randall St
...,...,...,...,...,...,...,...
10899,NaN,node-wtcst-wtctstairs-landing,World Trade Center,NaN,NaN,C1560,NaN
10900,NaN,node-wtcst-wtctstairs-top,World Trade Center,NaN,NaN,C1561,NaN
10901,NaN,place-buwst,Boston University West,NaN,NaN,C1562,NaN
10902,NaN,place-plsgr,Pleasant Street,NaN,NaN,C1563,NaN


In [8]:
# # Define the path for the updated CSV file in the same directory as the original lookup file
# output_csv_path = Path(original_stopid_lookup_file_directory).parent / "MBTA_Stopid_lookup_updated.csv"
output_csv_path = "./MBTA_Stopid_lookup_updated.csv"

# Export the updated lookup table without the index
df_updated_stopid_lookup.to_csv(output_csv_path, index=False)
print(f"Exported updated lookup table to: {output_csv_path}")


Exported updated lookup table to: ./MBTA_Stopid_lookup_updated.csv
